<a href="https://colab.research.google.com/github/Snowdenstyll/keno/blob/main/jupyterNotebooks/Keno_Lottery.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import numpy as np
import pandas as pd
from sklearn.preprocessing import StandardScaler

In [2]:
df = pd.read_csv("2024.csv")

In [3]:
df.head()
df.tail()
df.info()
df.describe()
df.drop(['PlayDate', 'AP'], axis=1, inplace=True)

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 130 entries, 0 to 129
Data columns (total 22 columns):
 #   Column    Non-Null Count  Dtype 
---  ------    --------------  ----- 
 0   PlayDate  130 non-null    object
 1   AP        130 non-null    object
 2   N01       130 non-null    int64 
 3   N02       130 non-null    int64 
 4   N03       130 non-null    int64 
 5   N04       130 non-null    int64 
 6   N05       130 non-null    int64 
 7   N06       130 non-null    int64 
 8   N07       130 non-null    int64 
 9   N08       130 non-null    int64 
 10  N09       130 non-null    int64 
 11  N10       130 non-null    int64 
 12  N11       130 non-null    int64 
 13  N12       130 non-null    int64 
 14  N13       130 non-null    int64 
 15  N14       130 non-null    int64 
 16  N15       130 non-null    int64 
 17  N16       130 non-null    int64 
 18  N17       130 non-null    int64 
 19  N18       130 non-null    int64 
 20  N19       130 non-null    int64 
 21  N20       130 no

In [4]:
scaler = StandardScaler().fit(df.values)
transformed_dataset = scaler.transform(df.values)
transformed_df = pd.DataFrame(data=transformed_dataset, index=df.index)

In [5]:
# All our games
number_of_rows = df.values.shape[0]
# Amount of games we need to take into consideration for prediction
window_length = 10
number_of_features = df.values.shape[1]

X = np.empty([ number_of_rows - window_length, window_length, number_of_features], dtype=float)
y = np.empty([ number_of_rows - window_length, number_of_features], dtype=float)


In [6]:
for i in range(0, number_of_rows-window_length):
    X[i] = transformed_df.iloc[i : i+window_length, 0 : number_of_features]
    y[i] = transformed_df.iloc[i+window_length : i+window_length+1, 0 : number_of_features]

In [7]:
# Recurrent Neural Netowrk (RNN) with Long Short Term Memory (LSTM)
# Importing the Keras libraries and packages
from keras.models import Sequential
from keras.layers import LSTM, Dense, Bidirectional, Dropout
batch_size = 70

# Initialising the RNN
model = Sequential()
# Adding the input layer and the LSTM layer
model.add(Bidirectional(LSTM(256,
                        input_shape = (window_length, number_of_features),
                        return_sequences = True)))
# Adding a first Dropout layer
model.add(Dropout(0.2))
# Adding a second LSTM layer
model.add(Bidirectional(LSTM(256,
                        input_shape = (window_length, number_of_features),
                        return_sequences = True)))
# Adding a second Dropout layer
model.add(Dropout(0.2))
# Adding a third LSTM layer
model.add(Bidirectional(LSTM(256,
                        input_shape = (window_length, number_of_features),
                        return_sequences = True)))
# Adding a fourth LSTM layer
model.add(Bidirectional(LSTM(256,
                        input_shape = (window_length, number_of_features),
                        return_sequences = False)))
# Adding a fourth Dropout layer
model.add(Dropout(0.2))
# Adding the first output layer
model.add(Dense(70))
# Adding the last output layer
model.add(Dense(number_of_features))

In [8]:
from tensorflow import keras
from tensorflow.keras.optimizers import Adam
from sklearn.model_selection import train_test_split

model.compile(optimizer=Adam(learning_rate=0.0001), loss ='mse', metrics=['accuracy'])

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.95, random_state=95)

# Train the model
history = model.fit(X, y, batch_size=batch_size, epochs=1000, verbose=2)
#history = model.fit(X_train, y_train, batch_size=100, epochs=600, verbose=2)

Epoch 1/1000
2/2 - 27s - loss: 1.0193 - accuracy: 0.0417 - 27s/epoch - 13s/step
Epoch 2/1000
2/2 - 0s - loss: 1.0152 - accuracy: 0.0417 - 60ms/epoch - 30ms/step
Epoch 3/1000
2/2 - 0s - loss: 1.0095 - accuracy: 0.0417 - 58ms/epoch - 29ms/step
Epoch 4/1000
2/2 - 0s - loss: 1.0059 - accuracy: 0.0917 - 57ms/epoch - 29ms/step
Epoch 5/1000
2/2 - 0s - loss: 1.0030 - accuracy: 0.0583 - 53ms/epoch - 26ms/step
Epoch 6/1000
2/2 - 0s - loss: 0.9981 - accuracy: 0.0917 - 51ms/epoch - 26ms/step
Epoch 7/1000
2/2 - 0s - loss: 0.9945 - accuracy: 0.0833 - 48ms/epoch - 24ms/step
Epoch 8/1000
2/2 - 0s - loss: 0.9873 - accuracy: 0.0917 - 53ms/epoch - 26ms/step
Epoch 9/1000
2/2 - 0s - loss: 0.9830 - accuracy: 0.1250 - 48ms/epoch - 24ms/step
Epoch 10/1000
2/2 - 0s - loss: 0.9772 - accuracy: 0.1250 - 44ms/epoch - 22ms/step
Epoch 11/1000
2/2 - 0s - loss: 0.9704 - accuracy: 0.1417 - 44ms/epoch - 22ms/step
Epoch 12/1000
2/2 - 0s - loss: 0.9618 - accuracy: 0.1000 - 47ms/epoch - 23ms/step
Epoch 13/1000
2/2 - 0s - l

In [9]:
loss, accuracy = model.evaluate(X, y, verbose=1)
print(f'Test Loss: {loss}, Test Accuracy: {accuracy}')

loss, accuracy = model.evaluate(X_test, y_test, verbose=1)
print(f'Test Loss: {loss}, Test Accuracy: {accuracy}')

4/4 [==============================] - 2s 11ms/step - loss: 0.0028 - accuracy: 0.9250
Test Loss: 0.00283616385422647, Test Accuracy: 0.925000011920929
4/4 [==============================] - 0s 10ms/step - loss: 0.0029 - accuracy: 0.9298
Test Loss: 0.0028514720033854246, Test Accuracy: 0.9298245906829834


In [10]:
#history = model.fit(X_train, y_train, batch_size=50, epochs=100, verbose=2)
#history = model.fit(X_train, y_train, batch_size=50, epochs=100, verbose=2)

In [11]:
df1 = df.copy()
to_predict = df1.tail(window_length)
to_predict.drop([to_predict.index[-1]],axis=0, inplace=True)
to_predict

<ipython-input-11-efb52fef0daf>:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  to_predict.drop([to_predict.index[-1]],axis=0, inplace=True)


,N01,N02,N03,N04,N05,N06,N07,N08,N09,N10,N11,N12,N13,N14,N15,N16,N17,N18,N19,N20
120,4,6,10,13,15,21,24,28,31,32,37,39,42,48,51,52,59,61,63,68
121,2,3,6,11,21,24,30,31,33,34,35,37,39,41,43,44,48,56,68,70
122,3,4,10,12,13,14,15,22,31,33,36,43,48,50,59,60,63,65,66,68
123,10,11,15,17,18,23,27,30,37,39,43,47,48,51,52,56,61,63,65,67
124,1,3,7,15,20,26,27,33,34,38,40,41,46,48,51,58,61,62,63,70
125,1,9,10,19,20,22,24,26,28,31,34,37,38,42,45,55,60,61,67,70
126,2,3,4,8,10,14,18,20,30,31,33,44,45,48,57,59,60,61,68,70
127,6,23,28,29,31,32,36,37,39,40,44,49,51,52,54,60,61,62,63,68
128,3,13,14,21,26,28,30,31,34,35,39,40,47,53,54,58,60,62,65,68


In [12]:
prediction = np.array(df.tail(1))

x_next = scaler.transform(to_predict)
y_pred = model.predict(np.array([x_next]))

temp = np.array(scaler.inverse_transform(y_pred).astype(int)[0])
combined_array = np.concatenate((temp, np.array(prediction[0])))
winning_numbers = np.bincount(combined_array)
winners = np.unique(combined_array)

uniques, uniq_idx, counts = np.unique(combined_array,return_index=True,return_counts=True)
duplicates = combined_array[ uniq_idx[counts>=2] ]  # <--- Get duplicates

top_5_indices = np.argsort(y_pred[0], axis=0)
prediction = scaler.inverse_transform(y_pred).astype(int)

top_5_numbers = []
for index in top_5_indices:
    # Access the corresponding number in the top_10_predictions array
    number = prediction[0, index]
    # Append the number to the top_5_numbers array
    top_5_numbers.append(number)

print("The top 5 indices", top_5_indices)
print("The top 5 numbers:", top_5_numbers)

print("The predicted numbers for the lottery game are:", scaler.inverse_transform(y_pred).astype(int)[0])
print("The actual numbers in the last lottery game were:", prediction[0])
print("\n")
print("winners:", duplicates)
print(len(duplicates)/20)


/usr/local/lib/python3.10/dist-packages/sklearn/base.py:432: UserWarning: X has feature names, but StandardScaler was fitted without feature names
  warnings.warn(


1/1 [==============================] - 6s 6s/step
The top 5 indices [ 0  1 18 11 12 19 10 13  2 14  7  8  6  3  9 15  4 17 16  5]
The top 5 numbers: [1, 5, 62, 39, 43, 67, 37, 48, 11, 51, 28, 31, 25, 16, 35, 56, 20, 62, 59, 23]
The predicted numbers for the lottery game are: [ 1  5 11 16 20 23 25 28 31 35 37 39 43 48 51 56 59 62 62 67]
The actual numbers in the last lottery game were: [ 1  5 11 16 20 23 25 28 31 35 37 39 43 48 51 56 59 62 62 67]


winners: [ 5 11 16 20 23 35 39 43 59 62 67]
0.55


# Predict the next set of numbers

In [13]:
next = df1.copy()
next = next.tail(window_length)
next = np.array(next)

x_next = scaler.transform(next)
y_next_pred = model.predict(np.array([x_next]))
prediction = scaler.inverse_transform(y_next_pred).astype(int)
print("The predicted numbers for the lottery game:", temp)
print("\n")
top_5_indices = np.argsort(y_next_pred[0], axis=0)[-5:]
top_5_numbers = []
for index in top_5_indices:
    # Access the corresponding number in the top_10_predictions array
    number = prediction[0, index]
    # Append the number to the top_5_numbers array
    top_5_numbers.append(number)


print("The top 5 indices", top_5_indices)
print("The top 5 numbers:", top_5_numbers)


1/1 [==============================] - 3s 3s/step
The predicted numbers for the lottery game: [ 1  5 11 16 20 23 25 28 31 35 37 39 43 48 51 56 59 62 62 67]


The top 5 indices [ 5 17  4 15 16]
The top 5 numbers: [22, 62, 19, 56, 59]


In [14]:
next = df1.copy()
next = next.tail(window_length)
next = np.array(next)

x_next = scaler.transform(next)
y_next_pred = model.predict(np.array([x_next]))

# Get the actual values of the top 10 predictions
prediction = scaler.inverse_transform(y_next_pred).astype(int)

# Get the indices of the top 10 predictions (change axis=0 if predictions are along a different axis)
top_10_indices = np.argsort(y_pred[0], axis=0)[-10:]
top_5_indices = np.argsort(y_pred[0], axis=0)[-5:]

top_10_numbers = []
top_5_numbers = []

# index [0,2,4] usually are correct
# Loop through each index in top_10_indices
for index in top_10_indices:
    # Access the corresponding number in the top_10_predictions array
    number = prediction[0, index]

    # Append the number to the top_10_numbers array
    top_10_numbers.append(number)

for index in top_5_indices:
    # Access the corresponding number in the top_10_predictions array
    number = prediction[0, index]

    # Append the number to the top_10_numbers array
    top_5_numbers.append(number)

print("The top 5 indices", top_5_indices)
print("The top 5 numbers:", top_5_numbers)
print("\n")

print("The top 10 indices", top_10_indices)
print("The top 10 numbers:", top_10_numbers)
print("\n")

print("The predicted numbers:", prediction)


1/1 [==============================] - 0s 23ms/step
The top 5 indices [15  4 17 16  5]
The top 5 numbers: [56, 19, 62, 59, 22]


The top 10 indices [ 7  8  6  3  9 15  4 17 16  5]
The top 10 numbers: [25, 29, 24, 15, 35, 56, 19, 62, 59, 22]


The predicted numbers: [[ 0  5 10 15 19 22 24 25 29 35 37 39 42 46 50 56 59 62 62 68]]
